In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession \
    .builder \
    .appName("concrec-rank") \
    .config("spark.driver.memory", "11g") \
    .getOrCreate()

21/11/11 17:13:27 WARN Utils: Your hostname, Daniels-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.5 instead (on interface en0)
21/11/11 17:13:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/11 17:13:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Load Data

In [3]:
from pyspark.sql.types import IntegerType

anime_df = spark.read.csv('../../data/anime/parsed_anime.csv', header=True, inferSchema=True)
anime_df = anime_df.withColumn('aired_from', col('aired_from').cast('int'))

# rename rating to all_rating
anime_df = anime_df.withColumnRenamed('rating', 'all_rating')

anime_df.printSchema()

root
 |-- anime_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- type: string (nullable = true)
 |-- episodes: string (nullable = true)
 |-- all_rating: double (nullable = true)
 |-- members: integer (nullable = true)
 |-- japanese_title: string (nullable = true)
 |-- aired: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- aired_from: integer (nullable = true)
 |-- aired_to: integer (nullable = true)



In [4]:
rating_df = spark.read.csv('../../data/anime/rating.csv', header=True, inferSchema=True)
rating_df = rating_df.filter(rating_df['rating'] > 0)
rating_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [5]:
like_threshold = 7.5

merged_df =  rating_df.join(
    anime_df.select('anime_id', 'name', 'genre', 'type', 'episodes', 
                    'all_rating', 'members', 'aired_from', 'aired_to'),
    on=['anime_id'], how='left'
).withColumn(
    'label',
    when(col('rating') >= like_threshold, 1).otherwise(0)
)

# Feature Builder

### 1. numeric features: min-max

In [6]:
from pyspark.ml.feature import MinMaxScaler, VectorAssembler
from pyspark.ml import Pipeline
import pyspark.sql.types as types
from pyspark.sql import functions as F

In [7]:
NUMBER_PRECISION = 2

@udf(types.FloatType())
def extract_float(l):
    r = __builtin__.round(l[0], NUMBER_PRECISION)
        
    return float(r)

In [8]:
class NumericScaler:
    def __init__(self, cols):
        self.cols = cols
        self.pipeline = self.__build_min_max_scalers(cols)
        
    def __build_min_max_scalers(self, cols):
        pipelines = [self.__build_one_min_max_scaler(col) for col in cols]
        return Pipeline(stages=pipelines)
        
    def __build_one_min_max_scaler(self, col):
        output_col = f"{col}_min_max"

        vec_assembler = VectorAssembler(inputCols=[col], outputCol=f"{col}_vec", handleInvalid='keep')
        min_max_scaler = MinMaxScaler(inputCol=f"{col}_vec", outputCol=output_col)
        pipeline = Pipeline(stages=[vec_assembler, min_max_scaler])

        return pipeline
        
    def fit(self, df):
        self.model = self.pipeline.fit(df)
        
    def transform(self, df):
        result = self.model.transform(df)
        
        # drop all intermedia cols and convert output to float
        for col in self.cols:
            output_col = f"{col}_min_max"
            result = result \
                .drop(f"{col}_vec") \
                .withColumn(output_col, extract_float(F.col(output_col)))
            
        return result

Anime related numeric features

In [9]:
item_numeric_cols = [
    'all_rating',
    'members',
    'aired_from',
    'aired_to'
]

item_numeric_scaler = NumericScaler(item_numeric_cols)

User related numeric features

In [10]:
user_numeric_cols = [
    'user_rating_ave',
    'user_rating_std',
    'user_aired_from_ave',
    'user_aired_to_ave'
]

In [11]:
user_numeric_scaler = NumericScaler(user_numeric_cols)

### 3. categorical data - multihot

In [12]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
import pyspark.sql.types as types
from pyspark.ml.linalg import SparseVector
import numpy as np

In [13]:
# 用已经训练好的string index mapping对genres数组进行encode
def encode_genres_col(index_mapping_broadcasted):
    @udf(returnType='array<int>')
    def encode_genres_col(genres, max_genre_index):
        if genres is None:
            genres = []
        gen_vec = [index_mapping_broadcasted.value.get(gen) for gen in genres]
        gen_vec = list(set(gen_vec)) # dedup

        # convert genre vector to multi-hot
        fill = np.ones(len(gen_vec), dtype=np.int32)
        sorted_index = np.sort(gen_vec)
        multihot_vec = SparseVector(max_genre_index + 1, sorted_index, fill)
        return multihot_vec.toArray().astype(np.int32).tolist()
    
    return encode_genres_col

In [14]:
class CategoricalEncoder:
    def __init__(self, colname):
        self.colname = colname
    
    def fit(self, df):
        exploded_df = df.withColumn(
            'genre_item',
            explode(col('genres'))
        )
        
        genre_string_indexer = StringIndexer(inputCol='genre_item', outputCol='genre_index')
        indexer_model = genre_string_indexer.fit(exploded_df)
        
        # get mapping from string indexer
        gens_df = spark.createDataFrame(
            [{'genre_item': g} for g in indexer_model.labels]
        )
        mapping_df = indexer_model.transform(gens_df).collect()
        mapping_dict = {row.genre_item: int(row.genre_index) for row in mapping_df}
        self.max_genre_index = __builtin__.max(mapping_dict.values())
        broadcasted = spark.sparkContext.broadcast(mapping_dict)
        
        self.encode_fn = encode_genres_col(broadcasted)
        
    def transform(self, df):
        return df \
            .withColumn(
                f"{self.colname}_multihot", 
                self.encode_fn(col(self.colname), lit(self.max_genre_index))
            )
        
        

Anime categorical encoder

In [15]:
item_categorical_encoder = CategoricalEncoder('genres')

User categorical encoder

In [16]:
user_categorical_encoder = CategoricalEncoder('user_liked_genres')

# Build Dataset for DNN Training

In [17]:
from pyspark.sql.window import Window

In [18]:
# 帮助方法：对于某一列，在聚合的时候，如果用户不喜欢这个电影，则不聚合这个电影的信息
likedMoviesCol = lambda colname: when(col('label') == 1, col(colname)).otherwise(lit(None))


@udf(returnType='array<string>')
def genre_to_list(gen_str):
    if gen_str is None:
        return []
    
    gens = gen_str.split(",")
    return [gen.strip() for gen in gens]


@udf(types.ArrayType(types.StringType()))
def most_liked_genres(gen_strs):
    """
    gen_strs = ["Action, Adventure, Drama", "Comedy, Drama, School"]
    """
    gens = [s.split(",") for s in gen_strs]
    gens = [x for l in gens for x in l] # flatten
    gens = [s.strip() for s in gens]
    
    gen_set = set(gens)
    count_occur = lambda gen, l: len([g for g in l if g == gen])
    gen_with_occur = [(gen, count_occur(gen, gens)) for gen in gen_set]
    gen_with_occur.sort(key=lambda x: x[1], reverse=True)
    
    # pick 5 most liked genres
    return [x[0] for x in gen_with_occur[:5]]

In [19]:
windowSpec = Window \
    .partitionBy('user_id') \
    .orderBy('aired_from') \
    .rowsBetween(-100, -1)

In [20]:
feat_df = merged_df \
    .withColumn('genres', genre_to_list(col('genre'))) \
    .withColumn('user_rating_cnt', count(lit(1)).over(windowSpec)) \
    .withColumn('user_rating_ave', mean(col('rating')).over(windowSpec)) \
    .withColumn('user_rating_ave', F.round(col('user_rating_ave'), NUMBER_PRECISION)) \
    .withColumn('user_rating_std', stddev(col('rating')).over(windowSpec)) \
    .withColumn('user_rating_std', F.round(col('user_rating_std'), NUMBER_PRECISION)) \
    .withColumn('user_aired_from_ave', mean(likedMoviesCol('aired_from')).over(windowSpec)) \
    .withColumn('user_aired_from_ave', F.round(col('user_aired_from_ave'), 0)) \
    .withColumn('user_aired_to_ave', mean(likedMoviesCol('aired_to')).over(windowSpec)) \
    .withColumn('user_aired_to_ave', F.round(col('user_aired_to_ave'), 0)) \
    .withColumn('user_liked_genres', most_liked_genres(collect_list(likedMoviesCol('genre')).over(windowSpec)))

In [21]:
feat_df.show(5, vertical=True)

-RECORD 0-----------------------------------
 anime_id            | 1090                 
 user_id             | 148                  
 rating              | 6                    
 name                | Mobile Suit Gundam I 
 genre               | Action, Adventure... 
 type                | Movie                
 episodes            | 1                    
 all_rating          | 7.43                 
 members             | 12877                
 aired_from          | 353347200            
 aired_to            | 353347200            
 label               | 0                    
 genres              | [Action, Adventur... 
 user_rating_cnt     | 0                    
 user_rating_ave     | null                 
 user_rating_std     | null                 
 user_aired_from_ave | null                 
 user_aired_to_ave   | null                 
 user_liked_genres   | []                   
-RECORD 1-----------------------------------
 anime_id            | 1091                 
 user_id  

# Train

1. Train item numeric

In [22]:
item_numeric_scaler.fit(feat_df)

Traceback (most recent call last):
  File "/Users/daniel/Programs/machine-learning/recsys-lecture/concrec/rank-service/venv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/Users/daniel/Programs/machine-learning/recsys-lecture/concrec/rank-service/venv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/daniel/Programs/machine-learning/recsys-lecture/concrec/rank-service/venv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/daniel/Programs/machine-learning/recsys-lecture/concrec/rank-service/venv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


2. Train user numeric

In [23]:
user_numeric_scaler.fit(feat_df)

3. Train item categorical

In [24]:
item_categorical_encoder.fit(feat_df)

/Users/daniel/Programs/machine-learning/recsys-lecture/concrec/rank-service/venv/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


4. Train user categorical

In [25]:
user_categorical_encoder.fit(feat_df)

# Transform

## Transform data for DNN Training

1. item & user numeric

In [26]:
transformed_df = item_numeric_scaler.transform(feat_df)
transformed_df = user_numeric_scaler.transform(transformed_df)

In [27]:
transformed_df.show(5, vertical=True)

21/11/11 17:15:34 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0-------------------------------------------
 anime_id                    | 1090                 
 user_id                     | 148                  
 rating                      | 6                    
 name                        | Mobile Suit Gundam I 
 genre                       | Action, Adventure... 
 type                        | Movie                
 episodes                    | 1                    
 all_rating                  | 7.43                 
 members                     | 12877                
 aired_from                  | 353347200            
 aired_to                    | 353347200            
 label                       | 0                    
 genres                      | [Action, Adventur... 
 user_rating_cnt             | 0                    
 user_rating_ave             | null                 
 user_rating_std             | null                 
 user_aired_from_ave         | null                 
 user_aired_to_ave           | null           

Traceback (most recent call last):
  File "/Users/daniel/Programs/machine-learning/recsys-lecture/concrec/rank-service/venv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/Users/daniel/Programs/machine-learning/recsys-lecture/concrec/rank-service/venv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/daniel/Programs/machine-learning/recsys-lecture/concrec/rank-service/venv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/daniel/Programs/machine-learning/recsys-lecture/concrec/rank-service/venv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


2. item & user categorical

In [28]:
transformed_df = item_categorical_encoder.transform(transformed_df)
transformed_df = user_categorical_encoder.transform(transformed_df)

## Transform data for Feature Serving

### 1. Item

In [29]:
anime_genres_df = anime_df \
    .withColumn('genres', genre_to_list(col('genre')))

1.1 numeric

In [30]:
item_numeric_transformed_df = item_numeric_scaler.transform(anime_genres_df)

Traceback (most recent call last):
  File "/Users/daniel/Programs/machine-learning/recsys-lecture/concrec/rank-service/venv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/Users/daniel/Programs/machine-learning/recsys-lecture/concrec/rank-service/venv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/daniel/Programs/machine-learning/recsys-lecture/concrec/rank-service/venv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/daniel/Programs/machine-learning/recsys-lecture/concrec/rank-service/venv/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


1.2 categorical

In [31]:
item_both_transformed_df = item_categorical_encoder.transform(item_numeric_transformed_df)

### 2. User

2.1 build user feature df

之前构造的feat_df已经包含每个用户最新的喜好数据了，      
只要选择feat_df中每个用户的最新数据即可，        
这里可以同样按照aired_from排序，构造window进行筛选

In [32]:
w = Window.partitionBy('user_id')

user_feat_df = feat_df.withColumn('max_aired', max('aired_from').over(w)) \
    .where(col('aired_from') == col('max_aired')) \
    .drop('max_aired') 

2.2 numeric

In [33]:
user_numeric_transformed_df = user_numeric_scaler.transform(user_feat_df)

In [34]:
# user_numeric_transformed_df.show(5, vertical=True)

2.3 categorical

In [35]:
user_both_transformed_df = user_categorical_encoder.transform(user_numeric_transformed_df)

# Save

## 1. Save transformed_df for DNN training

## 2. Save Item and User features to Redis

### 2.1 Collect both dataset

In [36]:
item_features = item_both_transformed_df.collect()

In [37]:
item_features[1]

Row(anime_id=5114, name='Fullmetal Alchemist: Brotherhood', genre='Action, Adventure, Drama, Fantasy, Magic, Military, Shounen', type='TV', episodes='64', all_rating=9.26, members=793665, japanese_title='鋼の錬金術師 FULLMETAL ALCHEMIST', aired='Apr 5, 2009 to Jul 4, 2010', image_url='https://cdn.myanimelist.net/images/anime/1223/96541.jpg', aired_from=1238860800, aired_to=1278172800, genres=['Action', 'Adventure', 'Drama', 'Fantasy', 'Magic', 'Military', 'Shounen'], all_rating_min_max=0.9900000095367432, members_min_max=0.7799999713897705, aired_from_min_max=0.8999999761581421, aired_to_min_max=0.8999999761581421, genres_multihot=[0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [38]:
user_features = user_both_transformed_df.collect()

In [39]:
user_features[1]

Row(anime_id=30276, user_id=463, rating=10, name='One Punch Man', genre='Action, Comedy, Parody, Sci-Fi, Seinen, Super Power, Supernatural', type='TV', episodes='12', all_rating=8.82, members=552458, aired_from=1443974400, aired_to=1450627200, label=1, genres=['Action', 'Comedy', 'Parody', 'Sci-Fi', 'Seinen', 'Super Power', 'Supernatural'], user_rating_cnt=74, user_rating_ave=8.46, user_rating_std=1.09, user_aired_from_ave=1168336312.0, user_aired_to_ave=1188773573.0, user_liked_genres=['Comedy', 'Romance', 'Drama', 'Shounen', 'Adventure'], user_rating_ave_min_max=0.8299999833106995, user_rating_std_min_max=0.17000000178813934, user_aired_from_ave_min_max=0.8999999761581421, user_aired_to_ave_min_max=0.9100000262260437, user_liked_genres_multihot=[1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### 2.2 Save to Redis

In [42]:
from redis import Redis

In [43]:
redis = Redis()

In [44]:
item_numeric_feature_prefix = 'rank:item:num'
user_numeric_feature_prefix = 'rank:user:num'
item_categorical_feature_prefix = 'rank:item:cat'
user_categorical_feature_prefix = 'rank:user:cat'

In [52]:
def save_numeric_features(rows, features, idcol, prefix):
    for row in rows:
        mapping = {feat: row[feat] for feat in features}
        key = f"{prefix}:{row[idcol]}"
        redis.hset(key, mapping=mapping)

2.2.1 Save item numeric features

In [53]:
item_num_features = [
    'all_rating_min_max',
    'members_min_max',
    'aired_from_min_max',
    'aired_to_min_max'
]

save_numeric_features(
    item_features, 
    item_num_features,
    'anime_id',
    item_numeric_feature_prefix
)

2.2.2 Save user numeric features

In [54]:
user_num_features = [
    'user_rating_ave_min_max',
    'user_rating_std_min_max',
    'user_aired_from_ave_min_max',
    'user_aired_to_ave_min_max'
]

save_numeric_features(
    user_features,
    user_num_features,
    'user_id',
    user_numeric_feature_prefix
)

In [63]:
import json

def list2str(l):
    return json.dumps(l)

def save_categorical_features(rows, feature, idcol, prefix):
    for row in rows:
        mapping = {
            feature: list2str(row[feature])
        }
        key = f"{prefix}:{row[idcol]}"
        redis.hset(key, mapping=mapping)

2.2.3 Save item categorical features

In [64]:
save_categorical_features(
    item_features,
    'genres_multihot',
    'anime_id',
    item_categorical_feature_prefix
)

2.2.4 Save user categorical features

In [67]:
save_categorical_features(
    user_features,
    'user_liked_genres_multihot',
    'user_id',
    user_categorical_feature_prefix
)